In [244]:
import numpy as np
import random
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from matplotlib import pyplot as plt
import seaborn as sns 
%matplotlib inline 
color = sns.color_palette()
sns.set_style('darkgrid')
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.neighbors import KNeighborsRegressor
import warnings
warnings.filterwarnings("ignore")

In [245]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [246]:
#Deleting outliers
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)
ntrain = train.shape[0]

In [247]:
y_train = np.log1p(train.pop("SalePrice")).values

In [248]:
all_data = pd.concat((train, test),axis =0)

print("all_data size is : {}".format(all_data.shape))

all_data size is : (2917, 79)


In [249]:
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
    
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
    
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
    
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

all_data = all_data.drop(['Utilities'], axis=1)

all_data["Functional"] = all_data["Functional"].fillna("Typ")

all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])

all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])

all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

In [250]:
#MSSubClass=The building class
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [251]:
# ExterQual_rep = {'Gd':3, 'TA':2, 'Ex':4, 'Fa':1}
# all_data.replace({"ExterQual": ExterQual_rep},inplace=True)

# ExterCond_rep = {'Gd':3, 'TA':2, 'Ex':4, 'Fa':1,'Po':0}
# all_data.replace({"ExterCond": ExterCond_rep},inplace=True)

# BsmtQual_rep = {'Gd':3, 'TA':2, 'Ex':4, 'Fa':1, 'None':0}
# all_data.replace({"BsmtQual": BsmtQual_rep},inplace=True)

# BsmtCond_rep = {'TA':3, 'Gd':4, 'None':0, 'Fa':2, 'Po':1}
# all_data.replace({"BsmtCond": BsmtCond_rep},inplace=True)

# BsmtExposure_rep = {'No':1, 'Gd':4, 'Mn':2, 'Av':3, 'None':0}
# all_data.replace({"BsmtExposure": BsmtExposure_rep},inplace=True)

# HeatingQC_rep = {'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0}
# all_data.replace({"HeatingQC": HeatingQC_rep},inplace=True)

# KitchenQual_rep = {'Gd':2, 'TA':1, 'Ex':3, 'Fa':0}
# all_data.replace({"KitchenQual": KitchenQual_rep},inplace=True)

# FireplaceQu_rep = {'None':0, 'TA':3, 'Gd':4, 'Fa':2, 'Ex':5, 'Po':1}
# all_data.replace({"FireplaceQu": FireplaceQu_rep},inplace=True)

# GarageQual_rep = {'TA':3, 'Fa':2, 'Gd':4, 'None':0, 'Ex':5, 'Po':1}
# all_data.replace({"GarageQual": GarageQual_rep},inplace=True)

# GarageCond_rep = {'TA':3, 'Fa':2, 'None':0, 'Gd':4, 'Po':1, 'Ex':5}
# all_data.replace({"GarageCond": GarageCond_rep},inplace=True)

# PoolQC_rep = {'None':0, 'Ex':3, 'Fa':1, 'Gd':2}
# all_data.replace({"PoolQC": PoolQC_rep},inplace=True)

In [221]:
# 'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual','FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond',
# 'BsmtExposure',
from sklearn.preprocessing import LabelEncoder
cols = ('BsmtFinType1', 'BsmtFinType2', 'Functional', 'Fence', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

Shape all_data: (2917, 78)


In [252]:
lbl = LabelEncoder() 
lbl.fit(list(all_data['FireplaceQu'].values)) 
test = lbl.transform(list(all_data['FireplaceQu'].values))

In [222]:
# Adding total sqfootage feature 
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [223]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

In [224]:
# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


Skew in numerical features: 



,Skew
MiscVal,21.939672
PoolQC,20.397039
PoolArea,17.688664
LotArea,13.109495
LowQualFinSF,12.084539
3SsnPorch,11.372080
LandSlope,4.973254
KitchenAbvGr,4.300550
BsmtFinSF2,4.144503
EnclosedPorch,4.002344


In [225]:
skewness = skewness[abs(skewness) > 0.05]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
#     plt.figure()
#     plt.subplot(1,2,1)
#     sns.distplot(all_data[feat])
#     plt.subplot(1,2,2)
#     sns.distplot(boxcox1p(all_data[feat], lam))
    all_data[feat] = boxcox1p(all_data[feat], lam)

There are 59 skewed numerical features to Box Cox transform


In [226]:
all_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalSF
Id,,,,,,,,,,,,,,,,,,,,,
1,2.885846,RL,5.831328,19.212182,0.730463,0.730463,1.540963,Lvl,Inside,0.0,...,0.0,0.0,1.820334,None,0.0,1.820334,1.194318,WD,Normal,14.976591
2,2.055642,RL,6.221214,19.712205,0.730463,0.730463,1.540963,Lvl,FR2,0.0,...,0.0,0.0,1.820334,None,0.0,2.440268,0.730463,WD,Normal,14.923100
3,2.885846,RL,5.914940,20.347241,0.730463,0.730463,0.000000,Lvl,Inside,0.0,...,0.0,0.0,1.820334,None,0.0,3.011340,1.194318,WD,Normal,15.149678
4,3.011340,RL,5.684507,19.691553,0.730463,0.730463,0.000000,Lvl,Corner,0.0,...,0.0,0.0,1.820334,None,0.0,1.820334,0.000000,WD,Abnorml,14.857121
5,2.885846,RL,6.314735,21.325160,0.730463,0.730463,0.000000,Lvl,FR2,0.0,...,0.0,0.0,1.820334,None,0.0,1.540963,1.194318,WD,Normal,15.852312


In [227]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2917, 220)


In [228]:
from sklearn.preprocessing import MinMaxScaler
all_data = MinMaxScaler().fit_transform(all_data)

In [229]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [230]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [231]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train)
    rmse= np.sqrt(-cross_val_score(model, train, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [232]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [233]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [234]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [243]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0007))
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.1139 (0.0060)



In [93]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 0.1114 (0.0074)



In [ ]:
KNN = make_pipeline(RobustScaler(), KNeighborsRegressor(n_neighbors=4, weights = 'distance',algorithm='brute', p=1, leaf_size=1))

score = rmsle_cv(KNN)
print("KNN score: {:.5f} ({:.5f})\n".format(score.mean(), score.std()))

In [ ]:
KRR = KernelRidge(alpha=0.4, kernel='polynomial', degree=4, coef0=4.5)
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [ ]:
averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
stacked_averaged_models.fit(train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred))

In [ ]:
model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

In [ ]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

In [ ]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = ensemble
sub.to_csv('submission.csv',index=False)